📘 Coffee Consumption Analysis – Capstone Project

💾 Installation

Install the required libraries:

In [ ]:
#pip install sqlalchemy
#pip install psycopg2-binary

+ sqlalchemy is the high-level SQL toolkit and Object Relational Mapper (ORM) for Python.

+ psycopg2-binary is the PostgreSQL database adapter required for sqlalchemy to connect to Postgres databases.

📦 Import Libraries

In [2]:
import pandas as pd
import os
from sqlalchemy import create_engine, text
from dotenv import dotenv_values

### 📊 Load FAO Coffee Supply Data

This dataset from **[FAOSTAT](https://www.fao.org/faostat/en/#data/FBS)** covers **coffee supply (in kg/capita/year)** for **every country between 2010 and 2022**.  
We'll use this data to understand national-level coffee consumption trends over time.

In [3]:
fao_df = pd.read_csv('data/FAOSTAT_data_en_7-24-2025.csv')
fao_df.head()

,Domain Code,Domain,Area Code (M49),Area,Element Code,Element,Item Code (FBS),Item,Year Code,Year,Unit,Value,Flag,Flag Description,Note
0,FBS,Food Balances (2010-),4,Afghanistan,645,Food supply quantity (kg/capita/yr),S2630,Coffee and products,2010,2010,kg/cap,0.00,E,Estimated value,NaN
1,FBS,Food Balances (2010-),4,Afghanistan,645,Food supply quantity (kg/capita/yr),S2630,Coffee and products,2011,2011,kg/cap,0.00,E,Estimated value,NaN
2,FBS,Food Balances (2010-),4,Afghanistan,645,Food supply quantity (kg/capita/yr),S2630,Coffee and products,2012,2012,kg/cap,0.00,E,Estimated value,NaN
3,FBS,Food Balances (2010-),4,Afghanistan,645,Food supply quantity (kg/capita/yr),S2630,Coffee and products,2013,2013,kg/cap,0.00,E,Estimated value,NaN
4,FBS,Food Balances (2010-),4,Afghanistan,645,Food supply quantity (kg/capita/yr),S2630,Coffee and products,2014,2014,kg/cap,0.01,E,Estimated value,NaN


### 🧬 Load NHANES Data

NHANES (National Health and Nutrition Examination Survey) is a program by the **[U.S. CDC](https://wwwn.cdc.gov/nchs/nhanes/default.aspx)**.  
It includes extensive health and dietary data collected in 2-year cycles.

Each data file is named to reflect its cycle. For example:

- `DEMO_C.XPT` → 2003–2004  
- `DEMO_G.XPT` → 2011–2012

You can refer to the suffix mapping table below to interpret the cycle:

| Suffix | Cycle                |
|--------|----------------------|
| _A     | 1999–2000            |
| _B     | 2001–2002            |
| _C     | 2003–2004            |
| _D     | 2005–2006            |
| _E     | 2007–2008            |
| _F     | 2009–2010            |
| _G     | 2011–2012            |
| _H     | 2013–2014            |
| _I     | 2015–2016            |
| _J     | 2017–2018            |
| _K     | 2019–2020 *(not released)* |
|P_     | 2017–Mar 2020 *(pre-pandemic)* |
| _L     | 2021–2023            |

Use the proper prefix (e.g., `P_`) for special cycles like 2017–2020 when loading files.


In [12]:
# NHANES cycle mapping
cycles = {
    '2009-2010': 'F',
    '2011-2012': 'G',
    '2013-2014': 'H',
    '2015-2016': 'I',
    '2017-2018': 'J',
    '2017-2020': 'P',  # pre-pandemic cycle with 'P_' prefix and no suffix
    '2021-2023': 'L'
}

def load_nhanes_file(cycle, suffix, file_prefix):
    if suffix == 'P':
        # Pre-pandemic files have 'P_' prefix and no suffix
        file_path = f'./data/nhanes_raw/P_{file_prefix}.xpt'
    else:
        # Other files follow the standard format
        file_path = f'./data/nhanes_raw/{file_prefix}_{suffix}.xpt'
    df = pd.read_sas(file_path, format='xport', encoding='utf-8')
    df['cycle'] = cycle
    return df

def load_all_cycles(base_prefix):
    all_dfs = []
    for cycle, suffix in cycles.items():
        try:
            df = load_nhanes_file(cycle, suffix, base_prefix)
            all_dfs.append(df)
        except FileNotFoundError:
            if suffix == 'P':
                print(f"Missing file: P_{base_prefix}.xpt")
            else:
                print(f"Missing file: {base_prefix}_{suffix}.xpt")
    return pd.concat(all_dfs, ignore_index=True)

# Load all required datasets
demo_all   = load_all_cycles('DEMO')
bmx_all    = load_all_cycles('BMX')
dr1tot_all = load_all_cycles('DR1TOT')
dr1iff_all = load_all_cycles('DR1IFF')

☕ Identify Coffee-Related Food Codes (FNDDS 2021–2023)

In [13]:
file_path = "../data/WWEIA_August2021_August2023_foodcat_FNDDS.xlsx"
xl = pd.ExcelFile(file_path)

# Load specific sheet
fndds = xl.parse(sheet_name='Aug2021-Aug2023_FNDDS_foodcat')

# Filter rows with 'coffee'
coffee_rows = fndds[fndds['category_description'].str.contains('coffee', case=False, na=False)]

# Get food_code list
coffee_codes = coffee_rows['food_code'].unique().tolist()

print(f"Found {len(coffee_codes)} coffee-related food codes.")
print(coffee_codes[:5])

Found 109 coffee-related food codes.
[92100000, 92100500, 92101000, 92101500, 92101600]


🎯 Filter NHANES Dietary Data for Coffee Consumers

In [14]:
# Filter individuals who consumed coffee
coffee_consumers = dr1iff_all[dr1iff_all['DR1IFDCD'].isin(coffee_codes)]
coffee_seqns = coffee_consumers['SEQN'].unique()

print(f"Number of individuals who consumed coffee: {len(coffee_seqns)}")

Number of individuals who consumed coffee: 21054


🧹 Subset All NHANES Tables for Coffee Consumers

In [15]:
coffee_caffeine = dr1tot_all[dr1tot_all['SEQN'].isin(coffee_seqns)]
coffee_demo     = demo_all[demo_all['SEQN'].isin(coffee_seqns)]
coffee_bmx      = bmx_all[bmx_all['SEQN'].isin(coffee_seqns)]

🧼 Clean and Deduplicate Data

In [16]:
coffee_consumers_clean = coffee_consumers.drop_duplicates(subset=['SEQN'])
coffee_consumers_clean_all = coffee_consumers.drop_duplicates(subset=['SEQN', 'DR1IFDCD'])

coffee_caffeine_clean = coffee_caffeine.drop_duplicates(subset='SEQN')
coffee_demo_clean     = coffee_demo.drop_duplicates(subset='SEQN')
coffee_bmx_clean      = coffee_bmx.drop_duplicates(subset='SEQN')

# Optional: Filter for specific variables (not shown here, uncomment if needed)
# coffee_demo_clean = coffee_demo_clean[['SEQN', 'RIDAGEYR', ...]]
# coffee_bmx_clean = coffee_bmx_clean[['SEQN', 'BMXBMI']]

In [17]:
coffee_consumers_clean

,SEQN,WTDRD1,WTDR2D,DR1ILINE,DR1DRSTZ,DR1EXMER,DRABF,DRDINT,DR1DBIH,DR1DAY,...,DR1IP182,DR1IP183,DR1IP184,DR1IP204,DR1IP205,DR1IP225,DR1IP226,cycle,WTDRD1PP,WTDR2DPP
0,51624.0,171886.802965,1.823029e+05,1.0,1.0,23.0,2.0,2.0,18.0,2.0,...,0.013,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2009-2010,NaN,NaN
49,51628.0,17509.779266,1.428635e+04,1.0,1.0,42.0,2.0,2.0,13.0,1.0,...,0.003,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2009-2010,NaN,NaN
80,51630.0,135969.226393,1.475223e+05,1.0,1.0,41.0,2.0,2.0,3.0,2.0,...,0.016,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2009-2010,NaN,NaN
95,51633.0,16307.201494,1.572519e+04,1.0,1.0,25.0,2.0,2.0,2.0,2.0,...,0.005,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2009-2010,NaN,NaN
131,51635.0,29638.114228,2.858032e+04,2.0,1.0,42.0,2.0,2.0,10.0,2.0,...,0.006,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2009-2010,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926971,142300.0,57178.292488,5.397605e-79,4.0,1.0,90.0,2.0,1.0,35.0,6.0,...,0.005,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2021-2023,NaN,NaN
926996,142301.0,16933.112349,1.825301e+04,4.0,1.0,73.0,2.0,2.0,16.0,4.0,...,0.005,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2021-2023,NaN,NaN
927019,142302.0,21406.169710,2.255082e+04,2.0,1.0,7.0,2.0,2.0,20.0,2.0,...,0.004,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2021-2023,NaN,NaN
927033,142303.0,45402.183153,4.894118e+04,3.0,1.0,7.0,2.0,2.0,37.0,5.0,...,0.002,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,2021-2023,NaN,NaN


🗄️ Export Cleaned Tables to PostgreSQL

In [ ]:
# Load credentials
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db   = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

# Create SQLAlchemy engine
engine = create_engine(f"postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}")

# Set search_path (schema)
with engine.connect() as conn:
    conn.execute(text(f"SET search_path TO {pg_schema}"))

# Export cleaned tables
coffee_consumers_clean.to_sql('coffee_consumers', con=engine, if_exists='replace', index=False, schema=pg_schema)
coffee_consumers_clean_all.to_sql('coffee_consumers_all', con=engine, if_exists='replace', index=False, schema=pg_schema)
coffee_caffeine_clean.to_sql('coffee_caffeine', con=engine, if_exists='replace', index=False, schema=pg_schema)
coffee_demo_clean.to_sql('coffee_demo', con=engine, if_exists='replace', index=False, schema=pg_schema)
coffee_bmx_clean.to_sql('coffee_bmx', con=engine, if_exists='replace', index=False, schema=pg_schema)
fao_df.to_sql('fao_coffee_supply', con=engine, if_exists='replace', index=False, schema=pg_schema)


✅ Summary

At this stage:

+ FAO coffee supply data is loaded and stored.

+ NHANES individuals who consumed coffee were identified.

+ Key demographic, intake, and BMI data were extracted and cleaned.

+ All processed data is saved to PostgreSQL for further analysis (e.g., in a dashboard or statistical modeling).
